In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import all the libraries

In [ ]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import zipfile

from tensorflow import keras
from tensorflow.keras import layers

from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

## Data Unzipping

In [ ]:
root_path = '../input/facial-keypoints-detection/'
csv_data = pd.read_csv(root_path + 'IdLookupTable.csv')
train_zip_dir = 'training.zip'
test_zip_dir = 'test.zip'

In [ ]:

#Will unzip the files so that you can see them..

with zipfile.ZipFile("../input/facial-keypoints-detection/"+train_zip_dir,"r") as z:
    z.extractall(".")

with zipfile.ZipFile("../input/facial-keypoints-detection/"+test_zip_dir,"r") as z2:
    z2.extractall(".")

## Data Analysing

In [ ]:
train_data = pd.read_csv('./training.csv')
copy_train_data = train_data.copy()

In [ ]:
train_data.isnull().any()
## so there are 3 columns which have no vacant values. so we will replace the missing values with median value of that particular column

### Replacing Nan with median for each column

In [ ]:
train_input = train_data.iloc[:,-1:]
train_output = train_data.iloc[:,:-1]

In [ ]:
train_output.fillna(train_output.median(), inplace= True)

### Now split the input and ouput where image is input and rest are the output

In [ ]:
imag = []
for i in range(0,len(train_input)):
    img = train_input['Image'][i].split(' ')
    img = [0 if x == '' else int(x) for x in img]
    imag.append(img)

In [ ]:
imag = np.array(imag)

In [ ]:
x_train = imag.reshape(-1,96,96,1)
plt.imshow(x_train[0],cmap='gray')

In [ ]:
y_train = []
for i in range(0,len(train_output)):
    y = train_output.iloc[i,:]
    y_train.append(y)
y_train = np.array(y_train,dtype = 'float')

In [ ]:
x_train = x_train/255

In [ ]:
input_shape = x_train.shape

In [ ]:
input_shape

In [ ]:
input_shape

### Now decide the model architecture

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=input_shape[1:]))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
model.fit(x_train,y_train,epochs = 50,batch_size = 128,validation_split = 0.2)

In [ ]:
loss = model.history.history['loss']
mae = model.history.history['mae']
plt.plot(loss)


In [ ]:
plt.plot(mae)

### Testing preparations

In [ ]:
test_input = pd.read_csv('./test.csv')

In [ ]:
test_imag = []
for i in range(0,len(test_input)):
    img = test_input['Image'][i].split(' ')
    img = [0 if x == '' else int(x) for x in img]
    test_imag.append(img)

In [ ]:
x_test = np.array(test_imag)
x_test = x_test.reshape(-1,96,96,1)
copy_x_test = x_test.copy()
x_test = x_test/255

In [ ]:
pred = model.predict(x_test)

In [ ]:
plt.imshow(copy_x_test[0], cmap='gray')

In [ ]:
image = x_test[0]
features_loc = np.array(pred[0], dtype = np.int)

for i in range(0,len(features_loc),2):
    x = features_loc[i]
    y = features_loc[i+1]
    
    cv2.circle(image, (x,y), radius=1, color=(0, 0, 255), thickness=-1)
    
    
    

In [ ]:
plt.imshow(image,cmap='gray')

we can see that model is working perfectly